In [27]:
%matplotlib inline

In [28]:
import xlrd, xlwt
import re
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [29]:
input_book = xlrd.open_workbook('social_demographic.xls')
input_sh = input_book.sheet_by_index(0)
output_book = xlwt.Workbook()
output_sh = output_book.add_sheet('Data')

In [30]:
i = 0
for rx in range(input_sh.nrows):
    if input_sh.cell_value(rx, 1):
        output_sh.write(i, 0, input_sh.cell_value(rx,1))
        for cx, cell in enumerate(input_sh.row(rx)[2:]):
            if cell.ctype in [0,1,6]:
                if cell.value not in ['...', '…', '-', '']:
                    s = cell.value
                    s = re.sub('[) ]', '', s)
                    s = re.sub('[,]', '.', s)
                    output_sh.write(i, cx+1, float(s))
            else:
                output_sh.write(i, cx+1, cell.value)
        i += 1

In [31]:
output_book.save('social_demographic_processed.xls')

In [32]:
data = pd.read_excel('social_demographic_processed.xls', header=0, index_col=0).dropna(axis=0, how='all').transpose()

In [33]:
data['consumer_price_index_1995'] = data['consumer_price_index'][4:] / 100
data['consumer_price_index_1995'][1995] = 1
data['consumer_price_index_1995'] = data['consumer_price_index_1995'].cumprod()

In [34]:
data['average_monthly_pension_indexed_kzt'] = data['average_monthly_pension_kzt'] / data['consumer_price_index_1995']
data['minimum_monthly_pension_indexed_kzt'] = data['minimum_monthly_pension_kzt'] / data['consumer_price_index_1995']
data['average_nominal_income_per_person_indexed_kzt'] = data['average_nominal_income_per_person_kzt'] / data['consumer_price_index_1995']
data['average_monthly_nominal_wage_indexed_kzt'] = data['average_monthly_nominal_wage_kzt'] / data['consumer_price_index_1995']
data['minimum_wage_indexed_kzt'] = data['minimum_wage_kzt'] / data['consumer_price_index_1995']

In [35]:
ed1 = pd.read_excel('middle_education_ratio.xls', header=0, index_col=0).transpose()
ed2 = pd.read_excel('higher_education_ratio.xls', header=0, index_col=0).transpose()
ed3 = pd.read_excel('elementary_education_ratio.xls', header=0, index_col=0).transpose()

In [36]:
hc = pd.read_excel('healthcare.xls', header=0, index_col=0).transpose()

In [37]:
merged = pd.concat([data, ed1, ed2, ed3, hc], axis=1)

In [38]:
merged

Year,population,expected_life_length_all,expected_life_length_men,expected_life_length_women,birth_rate_1000,death_rate_1000,child_death_rate_1000,mother_death_rate_1000,natural_population_increase,natural_population_increase_1000,...,minimum_wage_indexed_kzt,middle_education_ratio_total,middle_education_ratio_women,middle_education_ratio_men,higher_education_ratio_total,higher_education_ratio_women,higher_education_ratio_men,elementary_education_ratio_total,number_of_doctors_all_specs_k,number_of_paramedical_staff_k
1991,16358.222,67.60,62.60,72.40,21.50,8.20,27.30,67.2,218850.0,13.30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.1,198.9
1992,16451.711,67.40,62.40,72.30,20.50,8.40,25.90,76.8,200094.0,12.10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.4,196.3
1993,16426.478,65.40,60.10,70.80,19.30,9.50,28.10,62.8,159412.0,9.80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.5,189.5
1994,16334.865,64.90,59.70,70.30,18.90,9.90,27.10,69.4,145285.0,9.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61.1,177.7
1995,15956.667,63.50,58.00,69.40,17.50,10.70,27.00,77.3,107469.0,6.80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.1,168.4
1996,15675.819,63.60,58.00,69.70,16.30,10.70,25.40,69.4,87147.0,5.60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57.9,150.1
1997,15480.635,64.00,58.40,69.90,15.20,10.40,24.90,76.9,72218.0,4.80,...,2449.089679,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54.5,129.5
1998,15188.174,64.50,59.00,70.40,14.80,10.20,21.60,77.5,68066.0,4.60,...,2548.110088,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.2,120.4
1999,14955.106,65.63,60.60,70.90,14.57,9.87,20.40,65.3,70162.0,4.70,...,1975.645544,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.6,110.4
2000,14901.641,65.45,60.20,71.20,14.92,10.06,18.80,60.9,72276.0,4.80,...,2124.291903,91.35,92.55,90.18,37.76,41.29,34.32,99.958811,49.0,106.6


In [50]:
merged['hospital_beds_per_1000'] = (merged['hospital_beds_total']/merged['population'])*1000
merged['doctors_per_1000'] = (merged['number_of_doctors_all_specs_k']/merged['population'])*1000

In [51]:
merged.to_hdf('data.h5', key='social_demographic')